In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [4]:
#tail -n +2 train.csv | split -l 150000


In [5]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [6]:
path = 'train/%s' % (np.random.choice(splits))
#
columns = ['acoustic_data','time_to_failure']
df = pd.read_csv(path, float_precision='round_trip', header=None)
df.columns = columns

df.describe()

,acoustic_data,time_to_failure
count,150000.000000,150000.000000
mean,3.824493,5.271509
std,6.748668,0.011252
min,-158.000000,5.252496
25%,1.000000,5.262097
50%,4.000000,5.271597
75%,6.000000,5.281198
max,159.000000,5.290799


In [30]:
df['roll_1000'] = df['acoustic_data'].rolling(1000,min_periods=1000).mean()
df['roll_1000'].fillna(df['roll_1000'][1000],inplace=True)
df['shifted'] = df['roll_1000'].shift(1)
df['shifted'].fillna(df['roll_1000'][1000],inplace=True)
df["roll_diff"] = df['shifted'] - df['roll_1000']

In [32]:
df.sample(10)

,acoustic_data,time_to_failure,roll_1000,shifted,roll_diff
40751,2,5.280099,4.010,4.013,0.003
133457,9,5.256696,4.420,4.411,-0.009
87705,-6,5.268397,3.851,3.859,0.008
1073,1,5.290797,3.881,3.883,0.002
39585,4,5.281196,3.296,3.293,-0.003
67038,-2,5.273797,3.371,3.375,0.004
94815,4,5.266298,4.023,4.024,0.001
110551,0,5.262099,4.158,4.162,0.004
65101,10,5.273799,3.303,3.293,-0.010
73439,6,5.271599,3.609,3.610,0.001


In [33]:
TIMESTEPS=150

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

param_test = False
param_augment = False
param_scale = True
param_noise = 0.3
param_smart_augment = False


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,
                                                  param_scale,param_noise,param_smart_augment]) for chunk in split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0


Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.p

  File "/home/shemery/.local/lib/python3.6/site-packages/pandas/core/internals/managers.py", line 525, in fillna
    return self.apply('fillna', **kwargs)
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3826, in _quantile_unchecked
    interpolation=interpolation)
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/core/_methods.py", line 140, in _std
    keepdims=keepdims)
  File "/home/shemery/.local/lib/python3.6/site-packages/pandas/core/indexes/range.py", line 590, in __getitem__
    return super_getitem(key)
  File "/home/shemery/.local/lib/python3.6/site-packages/pandas/core/internals/managers.py", line 395, in apply
    applied = getattr(b, f)(**kwargs)
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3405, in _ureduce
    r = func(a, **kwargs)
KeyboardInterrupt


KeyboardInterrupt: 

  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3957, in _quantile_ureduce_func
    if np.any(n):
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/core/_methods.py", line 107, in _var
    arrmean = umr_sum(arr, axis, dtype, keepdims=True)
  File "/home/shemery/.local/lib/python3.6/site-packages/pandas/core/series.py", line 264, in __init__
    data = SingleBlockManager(data, index, fastpath=True)
KeyboardInterrupt
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py", line 2164, in any
    return _wrapreduction(a, np.logical_or, 'any', axis, None, out, keepdims=keepdims)
  File "/home/shemery/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py", line 86, in _wrapreduction
    return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
  File "/home/shemery/.local/lib/python3.6/site-packages/pandas/core/dtypes/missing.py", line 99, in isna
    return _isna(obj)
  File "/home/shemery/.local/lib/p

In [32]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

res[0].get()[100].head()

,median,mean,max,min,std,abs,q25,q50,q75,sum,uniq,pos,negs,ssum,imax,imin,abs_nrg,abs_sum_chg,autocorr_10,cid_ce,kurtosis,mean_chg,reocurring_pct,r_sigma,ratio_to_length,skewness,perc_reocurr_dp,perc_reocurr_all,sum_reoccurr_val,sum_reoccurr_dp,mean_head,mean_tail,abs_diff_head_tail,pos_head,neg_head,pos_tail,neg_tail,time_to_failure,augmented
0,0.271353,-0.377079,0.120424,0.758358,-0.376327,-0.295479,-0.871653,0.271353,-1.355096,-0.377079,1.416155,0.396607,-0.616796,-0.651108,1.233805,1.590727,-0.433728,-0.276684,0.461959,0.693694,-0.255244,-0.058170,-0.689959,0.074292,1.416155,1.843796,-0.689959,-1.671093,-0.286169,-1.323064,-0.377079,-0.377079,-0.295479,0.396607,-0.616796,0.396607,-0.616796,12.3846,True
1,0.644520,0.807959,-0.489440,0.674217,-0.259262,-0.592973,0.782012,0.644520,0.428832,0.807959,-0.092374,0.396607,-0.361218,0.107726,-0.318479,1.378324,-0.029995,0.061559,0.358731,0.460177,-0.537629,-1.301224,-0.516298,-1.072871,-0.092374,0.607727,-0.516298,-0.052746,-0.286169,0.780990,0.807959,0.807959,-0.592973,0.396607,-0.361218,0.396607,-0.361218,12.3846,True
2,-0.652686,1.530443,4.236824,-4.808376,5.271854,4.642148,-2.262502,-0.652686,2.324415,1.530443,3.204042,-3.005437,3.370229,4.794006,-0.289129,-0.735264,6.001797,4.830948,-1.621963,-3.519923,3.935571,-1.245221,4.227924,2.696381,3.204042,3.971153,4.227924,-1.970787,6.625531,1.198065,1.530443,1.530443,4.642148,-3.005437,3.370229,-3.005437,3.370229,12.3846,True
3,-2.731923,-1.959525,0.332519,0.194905,0.565162,0.091035,-2.262502,-2.731923,0.428832,-1.959525,0.689826,-2.065399,1.887873,-0.692444,1.178366,-1.626662,-0.162158,0.804567,-0.501126,-0.822958,-0.303026,-0.886873,0.217295,0.893695,0.689826,0.765001,0.217295,-0.652134,-0.245749,-2.182115,-1.959525,-1.959525,0.091035,-2.065399,1.887873,-2.065399,1.887873,12.3846,True
4,-1.165635,-0.413985,-0.489440,-0.043347,-0.205340,-0.250298,-0.782633,-1.165635,0.428832,-0.413985,1.136798,0.441370,-0.412333,-0.531792,-1.492476,-0.683034,-0.328044,-0.175025,-0.309157,0.120707,-0.362717,-0.584605,-0.049788,-0.089588,1.136798,0.519763,-0.049788,-1.191583,-0.084072,-1.024263,-0.413985,-0.413985,-0.250298,0.441370,-0.412333,0.441370,-0.412333,12.3846,True


In [8]:

import pickle

shared_train_data = pickle.load(open("shared_train_data.pickle", "rb"))
shared_val_data = pickle.load(open("shared_val_data.pickle", "rb"))

In [38]:
NUMBER_OF_FEATURES=len(res[0].get()[0].drop(['time_to_failure','segment','augmented'], axis=1).columns)

# training = np.empty((len(splits),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# targets = np.empty((len(splits),1),dtype=float)
# train_data = np.empty((len(shared_train_data),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# y_train = np.empty((len(shared_train_data),1),dtype=float)
# val_data = np.empty((len(shared_val_data),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# y_val = np.empty((len(shared_val_data),1),dtype=float)



training = []
targets = []
val_data = []
y_val = []


for r in res:
    for df in r.get():
#         if df['segment'].unique() in shared_train_data:
            training.append(df.drop(['time_to_failure','augmented'], axis=1).values)
            targets.append(np.array([df['time_to_failure'].unique()]))
#         elif df['segment'].unique() in shared_val_data: 
#             val_data.append(df.drop(['time_to_failure','segment','augmented'], axis=1).values)
#             y_val.append(np.array([df['time_to_failure'].unique()]))
            
training = np.array(training).reshape(-1,TIMESTEPS,NUMBER_OF_FEATURES)
targets = np.array(targets).reshape((len(targets),1))
# val_data = np.array(val_data).reshape((len(shared_val_data),TIMESTEPS,NUMBER_OF_FEATURES))
# y_val = np.array(y_val).reshape((len(shared_val_data),1))

ValueError: could not broadcast input array from shape (150,38) into shape (150)

In [ ]:
train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU#,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint


dropout=0.2
BATCH_SIZE=128


my_model = Sequential()


my_model.add(GRU(use_bias = True,#unit_forget_bias=True,\
                  units = 8,\
                  input_shape=(TIMESTEPS, NUMBER_OF_FEATURES),
                  dropout=dropout,recurrent_dropout=dropout,
                  #return_sequences=True
                  ))

#my_model.add(BatchNormalization())


                 
                    
#my_model.add(Dense(8))
#my_model.add(BatchNormalization())
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()

from keras.callbacks import Callback

filepath="lstm.weights.{epoch:02d}-{val_loss:.2f}.hdf5"


callbacks = [
    #EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
]


#my_model.load_weights('lstm.weights.393-2.15.hdf5')

history = my_model.fit(train_data, y_train, batch_size=BATCH_SIZE, epochs=1000, shuffle=True,
                      validation_data=(val_data,y_val), callbacks=callbacks
                    )


# import matplotlib.pyplot as plt


# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

In [14]:

columns=res[0].get()[0].columns.values

data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        data = data.append(df)
        i+=1
        

data.reset_index(drop=True,inplace=True)




In [15]:
preds = []

grouped = data[data['segment'].isin(shared_val_data)]\
        .drop(['time_to_failure', 'augmented'], axis=1)\
        .sort_values(by='segment').groupby('segment')


for name, group in grouped:
    preds.append(my_model.predict(group.drop(['segment'], axis=1).values.reshape(1,TIMESTEPS,NUMBER_OF_FEATURES))[0][0])

In [16]:
pickle.dump(preds, open( "stacked_lstm_val.pickle", "wb" ))


In [18]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [19]:
test_splits

['seg_b691c5.csv',
 'seg_a8e71a.csv',
 'seg_355717.csv',
 'seg_4e627b.csv',
 'seg_3fb24b.csv',
 'seg_0d0250.csv',
 'seg_7f3ab0.csv',
 'seg_272a47.csv',
 'seg_750e7e.csv',
 'seg_75dc3e.csv',
 'seg_c35940.csv',
 'seg_cd43b4.csv',
 'seg_7d88e3.csv',
 'seg_bdae02.csv',
 'seg_9c3ef8.csv',
 'seg_1bf7b3.csv',
 'seg_0dc38f.csv',
 'seg_bff392.csv',
 'seg_50b048.csv',
 'seg_507065.csv',
 'seg_f0db55.csv',
 'seg_0cdcc8.csv',
 'seg_687a5c.csv',
 'seg_566efe.csv',
 'seg_00e5f7.csv',
 'seg_a25c46.csv',
 'seg_8e5704.csv',
 'seg_6e572a.csv',
 'seg_5f352e.csv',
 'seg_a802fa.csv',
 'seg_cd2b34.csv',
 'seg_280e3b.csv',
 'seg_f383fb.csv',
 'seg_a56c04.csv',
 'seg_3f3983.csv',
 'seg_d516e3.csv',
 'seg_50e270.csv',
 'seg_170684.csv',
 'seg_f5d2dd.csv',
 'seg_ce9f59.csv',
 'seg_11647f.csv',
 'seg_2c69e4.csv',
 'seg_3452b2.csv',
 'seg_9bab11.csv',
 'seg_f8db0a.csv',
 'seg_01c775.csv',
 'seg_8cdf83.csv',
 'seg_b21273.csv',
 'seg_2ed79b.csv',
 'seg_81c1a6.csv',
 'seg_266805.csv',
 'seg_19e1ff.csv',
 'seg_259d65

In [20]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    test_res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300


In [22]:
ids = []
preds = []
i=0
for r in test_res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        #print(test_df.values.shape)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [25]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('lstm_preds.csv', index=False)

In [24]:
submission["time_to_failure"].describe()

count    2624.000000
mean     5.397872   
std      2.745490   
min     -0.016591   
25%      3.075488   
50%      4.849690   
75%      7.773914   
max      11.540263  
Name: time_to_failure, dtype: float64

In [52]:
submission["time_to_failure"].describe()

count    2624.000000
mean     5.322992   
std      2.626039   
min     -0.497457   
25%      3.176856   
50%      4.755400   
75%      7.700604   
max      10.682412  
Name: time_to_failure, dtype: float64

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()